In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [2]:
flights_df = pd.read_csv('Data/File_1.csv')
flights_df = flights_df.drop('Unnamed: 0', axis=1)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/File_1.csv'

In [ ]:
# Convert 'timestamp' and 'last_position' to datetime
flights_df['timestamp'] = pd.to_datetime(flights_df['timestamp'])
flights_df['last_position'] = pd.to_datetime(flights_df['last_position'])
flights_df

In [ ]:
flights_df.drop_duplicates(subset=['icao24', 'last_position'], inplace = True)
flights_df = flights_df[flights_df['baro_altitude'] >= 10000]
flights_df = flights_df[['timestamp', 'icao24', 'latitude', 'longitude', 'ground_speed', 'track', 'vertical_rate', 'baro_altitude']]

In [ ]:
flights_df

In [ ]:
flights_df.isnull().sum(), flights_df.shape

In [ ]:
flights_df.dropna(inplace = True)
flights_df.isnull().sum(), flights_df.shape

In [ ]:
flights_df_sorted = flights_df.sort_values(by=['icao24', 'timestamp'])
flights_df_sorted

In [ ]:
# Downsampling the dataset by keeping one in every 60 data points
downsampled_df = flights_df_sorted.iloc[::60, :]

In [ ]:
# Function to find the row with the closest timestamp to the target time
def find_first_future_row(df, target_time, icao24):
    future_rows = df[(df['icao24'] == icao24) & (df['timestamp'] >= target_time) & (df['timestamp'] <= target_time + timedelta(minutes=10))]
    return future_rows.iloc[0] if not future_rows.empty else None

# Testing the function on a small subset of the data
subset_df = downsampled_df

# Adding new columns for future position and altitude
subset_df['latitude_in_10min'] = np.nan
subset_df['longitude_in_10min'] = np.nan
subset_df['baro_altitude_in_10min'] = np.nan

# Iterating over each row in the subset
for index, row in subset_df.iterrows():
    future_time = row['timestamp'] + timedelta(minutes=10)
    future_row = find_first_future_row(subset_df, future_time, row['icao24'])
    if future_row is not None:
        subset_df.at[index, 'latitude_in_10min'] = future_row['latitude']
        subset_df.at[index, 'longitude_in_10min'] = future_row['longitude']
        subset_df.at[index, 'baro_altitude_in_10min'] = future_row['baro_altitude']

# Displaying the modified subset to check the results
subset_df.head()
subset_df.isna().sum()

In [ ]:
subset_df.dropna(inplace = True)

In [ ]:
subset_df

In [ ]:
subset_df.to_csv('Datasets/Cleaned_prepared_data.csv')